# 1) Eksik verilere kırılım ortalamaları atandı.
# 2) Aykırı gözlemler LOF ile belirlendi,10.değer sonrasına 10.değerin(eşik değerin) verileri atandı. 
# 3) Corr>0.9 olan değişkenlerden 1 tanesi bırakıldı kalanlar atıldı(14 değişken kaldı)
# 4) 322 veri ile; tune edilmiş ve edilmemiş, en iyi durumda:
- Regression rmse: 333
- Ridge rmse: 332
- Lasso rmse: 331
- Elastic Net rmse: 3330

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')


import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt


from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV



Hitters=pd.read_csv("../input/hitters/Hitters.csv")
df=Hitters.copy()
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sum()

In [ ]:
print("New League= A" ,df[df["NewLeague"]=="A"].agg({"Salary":"mean"}))
print("New League= N" ,df[df["NewLeague"]=="N"].agg({"Salary":"mean"}))
print("League= A" ,df[df["League"]=="A"].agg({"Salary":"mean"}))
print("League= N" ,df[df["League"]=="N"].agg({"Salary":"mean"}))
print("Division= E" ,df[df["Division"]=="E"].agg({"Salary":"mean"}))
print("Division= W" ,df[df["Division"]=="W"].agg({"Salary":"mean"}))

In [ ]:
df.loc[(df["Salary"].isnull())& (df['Division'] == 'E'),"Salary"]=624.27
df.loc[(df["Salary"].isnull())& (df['Division'] == 'W'),"Salary"]=450.87
df.isnull().sum().sum()

In [ ]:
# One hot Encoding
df = pd.get_dummies(df, columns = ['League', 'Division', 'NewLeague'], drop_first = True)
df.head()

In [ ]:
clf= LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(df)[0:10]

In [ ]:
df_scores=clf.negative_outlier_factor_
np.sort(df_scores)[0:20]

In [ ]:
sns.boxplot(df_scores);

In [ ]:
threshold=np.sort(df_scores)[10]
threshold

In [ ]:
df.loc[df_scores < threshold].index

In [ ]:
df.loc[df_scores == threshold]

In [ ]:
df.iloc[[82, 113, 217, 236, 249, 274, 295, 302, 314, 315],:]=[[495,151,17,61,84,78,10,5624,1679,275,884,1015,709,1045,88,13,2460.0,0,0,0]]

In [ ]:
df.iloc[82,:]

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
cor[cor>0.9]

In [ ]:
CRBI_cor=abs(cor["CRBI"])
CRBI_relevant_features = CRBI_cor[CRBI_cor>0.9]
CRBI_relevant_features

In [ ]:
df1=df.drop(["CAtBat","CHits","CHmRun","CRuns"], axis=1)
plt.figure(figsize=(12,10))
cor = df1.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
AtBat_cor=abs(cor["AtBat"])
AtBat_relevant_features = AtBat_cor[AtBat_cor>0.9]
AtBat_relevant_features

In [ ]:
df2=df1.drop(["AtBat","Runs"], axis=1)
plt.figure(figsize=(12,10))
cor = df2.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
df2.shape

In [ ]:
# Regression

y=df2["Salary"]
X=df2.drop("Salary", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=46)
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)
y_pred=reg_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Ridge Regression

ridge_model=Ridge().fit(X_train,y_train)
y_pred= ridge_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Ridge_tuned(alpha1)
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)
ridge_cv = RidgeCV(alphas = alphas1, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridge_cv.fit(X_train,y_train)
print(ridge_cv.alpha_)
ridge_tuned=Ridge(alpha=ridge_cv.alpha_).fit(X_train,y_train)
y_pred=ridge_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Ridge_tuned(alpha2)
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)
ridge_cv = RidgeCV(alphas = alphas2, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridge_cv.fit(X_train,y_train)
print(ridge_cv.alpha_)
ridge_tuned=Ridge(alpha=ridge_cv.alpha_).fit(X_train,y_train)
y_pred=ridge_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Ridge_tuned(alpha3)
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)
ridge_cv = RidgeCV(alphas = alphas3, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridge_cv.fit(X_train,y_train)
print(ridge_cv.alpha_)
ridge_tuned=Ridge(alpha=ridge_cv.alpha_).fit(X_train,y_train)
y_pred=ridge_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# Ridge_tuned(best alpha)
alphas4 = np.linspace(0,0.01,10)
ridge_cv = RidgeCV(alphas = alphas4, scoring = "neg_mean_squared_error", cv = 10, normalize = True)
ridge_cv.fit(X_train,y_train)
print(ridge_cv.alpha_)
ridge_tuned=Ridge(alpha=ridge_cv.alpha_).fit(X_train,y_train)
y_pred=ridge_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Lasso Regression
lasso_model = Lasso().fit(X_train, y_train)
y_pred=lasso_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Lasso Tuned(alpha1)
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)
lasso_cv_model = LassoCV(alphas = alphas1, cv = 10).fit(X_train, y_train)
print(lasso_cv_model.alpha_)
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
#Lasso Tuned(alpha2)
lasso_cv_model = LassoCV(alphas = alphas2, cv = 10).fit(X_train, y_train)
print(lasso_cv_model.alpha_)
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
#Lasso Tuned(alpha3)
lasso_cv_model = LassoCV(alphas = alphas3, cv = 10).fit(X_train, y_train)
print(lasso_cv_model.alpha_)
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
# Lasso_tuned(best alpha)
alphas4 = np.linspace(0,1,500)
lasso_cv_model = LassoCV(alphas = alphas4, cv = 10).fit(X_train, y_train)
print(lasso_cv_model.alpha_)
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_).fit(X_train, y_train)
y_pred = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

In [ ]:
# Elastic Net Regression
enet_model = ElasticNet().fit(X_train, y_train)
y_pred = enet_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Elastic Net(alpha1)
alphas1 = np.random.randint(0,1000,100)
alphas2 = 10**np.linspace(10,-2,100)*0.5
alphas3 = np.linspace(0,1,1000)
enet_cv_model = ElasticNetCV(alphas = alphas1, cv = 10).fit(X_train, y_train)
print(enet_cv_model.alpha_)
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Elastic Net(alpha2)
enet_cv_model = ElasticNetCV(alphas = alphas2, cv = 10).fit(X_train, y_train)
print(enet_cv_model.alpha_)
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
#Elastic Net(alpha3)
enet_cv_model = ElasticNetCV(alphas = alphas3, cv = 10).fit(X_train, y_train)
print(enet_cv_model.alpha_)
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
enet_params = {"l1_ratio": [0,0.01,0.05,0.1,0.2,0.4,0.5,0.6,0.8,1],
               "alpha":[0.1,0.01,0.04,0.05,0.001,0.5,0.9,1,5,10,20,50,87,99,100,110,194,195,200,265,266,289,325,330,350,352,355,400]}
enet_model = ElasticNet().fit(X, y)
from sklearn.model_selection import GridSearchCV
gs_cv_enet = GridSearchCV(enet_model, enet_params, cv = 10).fit(X, y)
print(gs_cv_enet.best_params_)
enet_tuned = ElasticNet(**gs_cv_enet.best_params_).fit(X_train, y_train)
y_pred = enet_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))